# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [27]:
import numpy as np
import os
import datetime
import random as rn
from keras import backend as K
import tensorflow as tf
import imageio.v2 as imageio

We set the random seed so that the results don't vary drastically.

In [28]:
rn.seed(30)
np.random.seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [29]:
train_doc = np.random.permutation(open('../../datasets/videos/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('../../datasets/videos/Project_data/val.csv').readlines())
batch_size = 32

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [30]:
def generator(source_path, folder_list, batch_size):
    
    _height, _width, _step_size = 100, 100, 2
    print( 'Source path = ', source_path, '; Batch size = ', batch_size)
    
    img_idx = list(range(0, 30, _step_size))
    
    while True:
        
        t = np.random.permutation(folder_list)
        
        num_batches = len(folder_list) // batch_size
        
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),_height,_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    img_path = source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]
                    image = imageio.imread(img_path).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape
                    image = image/255
                    resize_image = tf.image.resize(image, [_height, _width])
                    #image = image[int(h/2)-50:int(h/2)+50, int(w/2)-50:int(w/2)+50, : ]
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder, idx] = resize_image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        num_stray_batches = len(folder_list) % batch_size
        
        for batch in range(num_stray_batches): # we iterate over the number of batches
            batch_data = np.zeros((num_stray_batches,len(img_idx),_height,_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((num_stray_batches,5)) # batch_labels is the one hot representation of the output
            for folder in range(num_stray_batches): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*num_stray_batches)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    img_path = source_path+'/'+ t[folder + (batch*num_stray_batches)].strip().split(';')[0]+'/'+imgs[item]
                    image = imageio.imread(img_path).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape
                    image = image/255
                    resize_image = tf.image.resize(image, [_height, _width])
                    #image = image[int(h/2)-50:int(h/2)+50, int(w/2)-50:int(w/2)+50, : ]
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder, idx] = resize_image
                    
                batch_labels[folder, int(t[folder + (batch*num_stray_batches)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [31]:
curr_dt_time = datetime.datetime.now()
train_path = '../../datasets/videos/Project_data/train'
val_path = '../../datasets/videos/Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 10
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, ConvLSTM2D, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

model = Sequential([
    ConvLSTM2D(
        filters=32,
        kernel_size=(3, 3),
        input_shape=(15, 100, 100, 3),  # (num_frames, height, width, channels)
        padding='same',
        activation='relu',
        return_sequences=False  # Set to True if stacking multiple ConvLSTM layers
    ),
    Flatten(),  # Flatten the 3D output to 1D for Dense layers
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Helps stabilize training
    Dense(5, activation='softmax')  # Output layer for 5 classes
])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [33]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d_5 (ConvLSTM2D)      │ (None, 100, 100, 32)   │        40,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 320000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │    40,960,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,001,221 (156.41 MB)

 Trainable params: 41,001,221 (156.41 MB)

 Non-trainable params: 0 (0.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')


LR = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5,
           patience=1, verbose=1)

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [36]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [37]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  ../../datasets/videos/Project_data/train ; Batch size =  32
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - categorical_accuracy: 0.2132 - loss: 7.1429Source path =  ../../datasets/videos/Project_data/val ; Batch size =  32

Epoch 1: saving model to model_init_2024-12-2906_18_03.993018/model-00001-4.18030-0.22775-1.57731-0.27000.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - categorical_accuracy: 0.2139 - loss: 7.0083 - val_categorical_accuracy: 0.2700 - val_loss: 1.5773 - learning_rate: 0.0010
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - categorical_accuracy: 0.3198 - loss: 1.5474
Epoch 2: saving model to model_init_2024-12-2906_18_03.993018/model-00002-1.52620-0.33540-1.41506-0.52000.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 96s 5s/step - categorical_accuracy: 0.3205 - loss: 1.5464 - val_categorical_accuracy: 0.5200 - val_loss: 1.4151 - learning_rate: 0.0010
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - categorical_accuracy: 0.5116 - loss: 1.3720
Epoch 3: saving 